In [1]:
import torch
import torchaudio
from IPython.display import clear_output
import gc

In [2]:
torchaudio.set_audio_backend("soundfile")

/tmp/ipykernel_489055/1822137616.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks/swishnet')
# import sys
# sys.path.append(os.getcwd())

In [4]:
sampling_rate = 16000
cutsize = 32000
suffix = "_16K_32000cut"

In [5]:
def collect_wav_files(base_dir):
    collected = []
    for root, _, files in os.walk(base_dir):  # 모든 하위 디렉토리 탐색
        for fname in files:
            if fname.lower().endswith(".wav"):  # 확장자 필터
                full_path = os.path.join(root, fname)
                collected.append(full_path)
    return collected


In [6]:
unpreprocessed_car_dir = "./car"
unpreprocessed_music_dir = "./music"
unpreprocessed_noise_dir = "./noise"
unpreprocessed_speech_dir = "./speech"

In [7]:
unpreprocessed_car_list = collect_wav_files(unpreprocessed_car_dir)
unpreprocessed_music_list = collect_wav_files(unpreprocessed_music_dir)
unpreprocessed_noise_list = collect_wav_files(unpreprocessed_noise_dir)
unpreprocessed_speech_list = collect_wav_files(unpreprocessed_speech_dir)

In [8]:
def get_wav_info_torchaudio(filepath):
    info = torchaudio.info(filepath)
    
    print(f"파일 경로: {filepath}")
    print(f"샘플링 레이트 (Hz): {info.sample_rate}")
    print(f"채널 수: {info.num_channels}")
    print(f"총 프레임 수: {info.num_frames}")
    print(f"재생 시간 (초): {info.num_frames / info.sample_rate:.2f}")
    print(f"비트 깊이: {info.bits_per_sample}")

In [9]:
print(unpreprocessed_car_list[0])
get_wav_info_torchaudio(unpreprocessed_car_list[0])


./car/vehicle_vehicle/bus/Bus_Driving_Sound_1_HOUR_650.wav
파일 경로: ./car/vehicle_vehicle/bus/Bus_Driving_Sound_1_HOUR_650.wav
샘플링 레이트 (Hz): 44100
채널 수: 2
총 프레임 수: 220500
재생 시간 (초): 5.00
비트 깊이: 16


In [10]:
resamplers = {}

def preprocess(preprocessed_data_list):
    data_list = []
    for path in preprocessed_data_list:
        try:
            print(f" Processing: {path}")
            waveform, sample_rate = torchaudio.load(path)

            if sample_rate != sampling_rate:        #만약 원하는 샘플링 레이트랑 다를 경우
                if sample_rate not in resamplers:   #특정 리샘플러가 없을 경우
                    resamplers[sample_rate] = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=sampling_rate) #리샘플러 딕셔너리에 sample_rate 의 이름으로 리샘플러 생성
                waveform = resamplers[sample_rate](waveform)  #sample_rate 리샘플러 불러와서 리샘플

            if waveform.shape[0] > 1: #만약 채널이 2이상일경우
                waveform = waveform.mean(dim=0, keepdim=True) #두 채널을 평균낸다
            waveform = waveform.squeeze(0)

            total_length = waveform.shape[0]
            valid_length = (total_length // cutsize) * cutsize #컷팅사이즈 정수배로 잘라 유효한 길이 

            for i in range(valid_length // cutsize): #커팅사이즈 길이만큼
                try:
                    segment = waveform[i * cutsize : (i + 1) * cutsize] #waveform의 커팅사이즈 i ~ i+1만큼을 
                    data_list.append(segment) 
                except Exception as seg_error:
                    print(f"⚠️ Error cutting segment {i} of {path}: {seg_error}")
                    continue
            del waveform, segment
            gc.collect()
        except Exception as e:
            print(f" Failed to process {path}: {e}")
            continue
    clear_output(wait=True)
    return data_list


In [11]:
# car_list = preprocess(unpreprocessed_car_list)
# print(len(car_list))
# torch.save(car_list, f"../unsplited/car/car_data{suffix}.pt")

In [12]:
# music_list = preprocess(unpreprocessed_music_list)
# print(len(music_list))
# torch.save(music_list, f"../unsplited/music/music_data{suffix}.pt")

In [13]:
# noise_list = preprocess(unpreprocessed_noise_list)
# print(len(noise_list))
# torch.save(noise_list, f"../unsplited/noise/noise_data{suffix}.pt")

In [14]:
speech_list = preprocess(unpreprocessed_speech_list)
print(len(speech_list))
torch.save(speech_list, f"../unsplited/speech/speech_data{suffix}.pt")

108561
